# FEMA DATA

This dataset is used to determine the houses that was damaged by flood and the amount of claim that was paid by FEMA to damaged house. There are additional information for each house including lat, long, year the house was built, number of stories, city, zip code, etc. 

The scope if to find another dataset from NASA that includes percipitation. Then combine the 2 dataset and train the model to predict the expected "claim" based on "percipitation". Additional data will be used to better train the model including location, year that the building was built, etc. 

## A. READ FEMA DATA from CSV FILE

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

# Import our input dataset
FEMA_df = pd.read_csv('../FIMA_NFIP_Redacted_Claims_Data_Set/openFEMA_claims20190831.csv')
FEMA_df.head()

C:\Users\hcivi_000.000\Anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,11,15,22,23,31,32,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,agriculturestructureindicator,asofdate,basefloodelevation,basementenclosurecrawlspacetype,reportedcity,condominiumindicator,policycount,countycode,crsdiscount,dateofloss,...,amountpaidonincreasedcostofcomplianceclaim,postfirmconstructionindicator,ratemethod,smallbusinessindicatorbuilding,state,totalbuildinginsurancecoverage,totalcontentsinsurancecoverage,yearofloss,reportedzipcode,primaryresidence
0,NaN,2019-08-31,NaN,0.0,OCEANSIDE,N,1.0,6073.0,0.00,1998-02-07,...,NaN,N,7,NaN,CA,200000.0,50000.0,1998,92056,NaN
1,NaN,2019-08-31,NaN,0.0,NEW ORLEANS,N,1.0,22071.0,0.00,2005-08-29,...,NaN,N,7,NaN,LA,100000.0,40000.0,2005,70131,Y
2,NaN,2019-08-31,NaN,0.0,NAVARRE,N,1.0,12113.0,0.05,1998-09-28,...,0.0,N,1,NaN,FL,100000.0,50000.0,1998,32566,NaN
3,NaN,2019-08-31,NaN,1.0,BEAUFORT,N,1.0,45013.0,0.00,1994-10-07,...,NaN,N,7,NaN,SC,100000.0,25000.0,1994,29902,NaN
4,NaN,2019-08-31,NaN,0.0,MELBOURNE,N,1.0,12009.0,0.00,1996-03-11,...,NaN,Y,7,NaN,FL,100000.0,25000.0,1996,32940,NaN


## B. Remove Unnecessary Columns From FEMA Dataset

### B.1. Remove columns that are obviously not adding any value

In [2]:
FEMA_Clean_df=FEMA_df.drop(columns=["policycount","countycode","crsdiscount","condominiumindicator","agriculturestructureindicator",
                                    "basementenclosurecrawlspacetype",'asofdate',"amountpaidoncontentsclaim","obstructiontype",
                      'basefloodelevation',"elevationcertificateindicator","elevatedbuildingindicator","censustract",
                      "houseworship","locationofcontents","lowestadjacentgrade","postfirmconstructionindicator","yearofloss",
                      "lowestfloorelevation","nonprofitindicator","originalnbdate","amountpaidonincreasedcostofcomplianceclaim",
                      "smallbusinessindicatorbuilding","primaryresidence","ratemethod","totalbuildinginsurancecoverage","totalcontentsinsurancecoverage"])
FEMA_Clean_df.head(10)

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
0,OCEANSIDE,1998-02-07,999.0,X,33.2,-117.3,4.0,1.0,1963-01-01,0.00,CA,92056
1,NEW ORLEANS,2005-08-29,999.0,X,29.9,-90.0,2.0,1.0,1967-07-01,0.00,LA,70131
2,NAVARRE,1998-09-28,999.0,X,30.4,-86.9,1.0,1.0,1972-01-01,8813.21,FL,32566
3,BEAUFORT,1994-10-07,999.0,X,32.4,-80.7,2.0,1.0,1960-01-01,2906.00,SC,29902
4,MELBOURNE,1996-03-11,999.0,X,28.3,-80.7,1.0,1.0,1988-01-01,3875.53,FL,32940
5,VIRGINIA BEACH,1998-02-03,999.0,AE,36.9,-76.0,1.0,1.0,1970-01-01,14985.92,VA,23451
6,HOUSTON,2017-08-27,999.0,X,29.7,-95.5,1.0,1.0,1974-01-01,195857.43,TX,77096
7,WAIANAE,1992-09-11,999.0,VE,21.4,-158.2,2.0,1.0,1974-07-31,0.00,HI,96792
8,MARATHON,1998-09-28,999.0,AE,24.7,-81.1,2.0,3.0,1973-01-01,2389.00,FL,33050
9,CARMEL,1995-03-11,999.0,AE,36.5,-121.9,1.0,1.0,1957-01-01,12737.86,CA,93923


In [3]:
FEMA_Clean_df.shape

(2432888, 12)

In [4]:
FEMA_Clean_df.dtypes

reportedcity                           object
dateofloss                             object
elevationdifference                   float64
floodzone                              object
latitude                              float64
longitude                             float64
numberoffloorsintheinsuredbuilding    float64
occupancytype                         float64
originalconstructiondate               object
amountpaidonbuildingclaim             float64
state                                  object
reportedzipcode                        object
dtype: object

### B.2. Remove Columnms that has many empty cells and check to see remaining columns dont have many empty cells

In [5]:
#Remove all Data that don’t have an City defined.
FEMA_Clean1_df=FEMA_Clean_df[FEMA_Clean_df["reportedcity"] != "N/A"]
FEMA_Clean1_df.dropna(subset=['reportedcity'], how='all', inplace=True)
print(FEMA_Clean1_df.shape)

FEMA_Clean2_df=FEMA_Clean1_df[FEMA_Clean1_df["amountpaidonbuildingclaim"] != "N/A"]
FEMA_Clean2_df.dropna(subset=['amountpaidonbuildingclaim'], how='all', inplace=True)
print(FEMA_Clean2_df.shape)

FEMA_Clean3_df=FEMA_Clean2_df[FEMA_Clean2_df["originalconstructiondate"] != "N/A"]
FEMA_Clean3_df.dropna(subset=['originalconstructiondate'], how='all', inplace=True)
print(FEMA_Clean3_df.shape)

FEMA_Clean4_df=FEMA_Clean3_df[FEMA_Clean3_df["floodzone"] != "N/A"]
FEMA_Clean4_df.dropna(subset=['floodzone'], how='all', inplace=True)
print(FEMA_Clean4_df.shape)

FEMA_Clean5_df=FEMA_Clean4_df[FEMA_Clean4_df["occupancytype"] != "N/A"]
FEMA_Clean5_df.dropna(subset=['occupancytype'], how='all', inplace=True)
print(FEMA_Clean5_df.shape)

FEMA_Clean6_df=FEMA_Clean5_df[FEMA_Clean5_df["numberoffloorsintheinsuredbuilding"] != "N/A"]
FEMA_Clean6_df.dropna(subset=['numberoffloorsintheinsuredbuilding'], how='all', inplace=True)
print(FEMA_Clean6_df.shape)

FEMA_Clean7_df=FEMA_Clean6_df[FEMA_Clean6_df["latitude"] != "N/A"]
FEMA_Clean7_df.dropna(subset=['latitude'], how='all', inplace=True)
print(FEMA_Clean7_df.shape)

FEMA_Clean8_df=FEMA_Clean7_df[FEMA_Clean7_df["occupancytype"] != "N/A"]
FEMA_Clean8_df.dropna(subset=['longitude'], how='all', inplace=True)
print(FEMA_Clean8_df.shape)

FEMA_Clean9_df=FEMA_Clean8_df[FEMA_Clean8_df["dateofloss"] != "N/A"]
FEMA_Clean9_df.dropna(subset=['dateofloss'], how='all', inplace=True)
print(FEMA_Clean9_df.shape)

FEMA_Clean10_df=FEMA_Clean9_df[FEMA_Clean9_df["reportedzipcode"] != "N/A"]
FEMA_Clean10_df.dropna(subset=['reportedzipcode'], how='all', inplace=True)
print(FEMA_Clean10_df.shape)

FEMA_Clean11_df=FEMA_Clean10_df[FEMA_Clean10_df["reportedzipcode"] != "N/A"]
FEMA_Clean11_df.dropna(subset=['reportedzipcode'], how='all', inplace=True)
print(FEMA_Clean11_df.shape)

FEMA_Clean12_df=FEMA_Clean11_df[FEMA_Clean11_df["elevationdifference"] != "N/A"]
FEMA_Clean12_df.dropna(subset=['elevationdifference'], how='all', inplace=True)
print(FEMA_Clean12_df.shape)

FEMA_Clean13_df=FEMA_Clean12_df[FEMA_Clean12_df["state"] != "N/A"]
FEMA_Clean13_df.dropna(subset=['state'], how='all', inplace=True)
print(FEMA_Clean13_df.shape)

(2428382, 12)


C:\Users\hcivi_000.000\Anaconda3\envs\mlenv\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


(2355202, 12)
(2021408, 12)
(2001194, 12)
(2001175, 12)
(2001173, 12)
(1986473, 12)
(1986473, 12)
(1986473, 12)
(1986351, 12)
(1986351, 12)
(1986351, 12)
(1986351, 12)


### B.3. Check to make sure there is no empty cells in the dataframe

In [6]:
# Drop rows with any empty cells, check
FEMA_Clean14_df=FEMA_Clean13_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
print(FEMA_Clean14_df.shape)
FEMA_Clean14_df

(1986351, 12)


,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
0,OCEANSIDE,1998-02-07,999.0,X,33.2,-117.3,4.0,1.0,1963-01-01,0.00,CA,92056
1,NEW ORLEANS,2005-08-29,999.0,X,29.9,-90.0,2.0,1.0,1967-07-01,0.00,LA,70131
2,NAVARRE,1998-09-28,999.0,X,30.4,-86.9,1.0,1.0,1972-01-01,8813.21,FL,32566
3,BEAUFORT,1994-10-07,999.0,X,32.4,-80.7,2.0,1.0,1960-01-01,2906.00,SC,29902
4,MELBOURNE,1996-03-11,999.0,X,28.3,-80.7,1.0,1.0,1988-01-01,3875.53,FL,32940
...,...,...,...,...,...,...,...,...,...,...,...,...
2432883,MIAMI,2005-08-26,1.0,AHB,25.6,-80.4,1.0,1.0,1985-01-01,4390.96,FL,33177
2432884,MIAMI,2005-08-25,1.0,AHB,25.6,-80.4,1.0,1.0,1990-01-01,3541.59,FL,33186
2432885,PINECREST,2005-06-20,-1.0,AE,25.7,-80.3,1.0,1.0,1959-01-01,8335.35,FL,33156
2432886,PINECREST,2005-08-25,999.0,AE,25.7,-80.3,1.0,1.0,1959-01-01,2396.44,FL,33156


## C. Check FEMA DataFrame Data-types and convert dates to DateTime

In [7]:
FEMA_Clean14_df.dtypes

reportedcity                           object
dateofloss                             object
elevationdifference                   float64
floodzone                              object
latitude                              float64
longitude                             float64
numberoffloorsintheinsuredbuilding    float64
occupancytype                         float64
originalconstructiondate               object
amountpaidonbuildingclaim             float64
state                                  object
reportedzipcode                        object
dtype: object

In [8]:
#convert dateofloss type to datetime
FEMA_Clean14_df['dateofloss'] =  pd.to_datetime(FEMA_Clean14_df['dateofloss'], format='%Y-%m-%d')
FEMA_Clean14_df.dtypes

reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate                      object
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
dtype: object

In [9]:
#Identify which row makes error on originalconstructiondate and doesnt let to change the type to date
FEMA_Clean14_df['originalconstructiondate'] =  pd.to_datetime(FEMA_Clean14_df['originalconstructiondate'], errors='coerce')
FEMA_Clean14_df.sort_values('originalconstructiondate')

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
180884,CHARLESTON,2015-10-05,-9.0,AE,32.8,-79.9,3.0,1.0,1860-01-01,20346.88,SC,29401
1240464,CHARLESTON,2016-10-08,999.0,AE,32.8,-79.9,3.0,6.0,1876-07-01,36988.33,SC,29401
1717766,SAVANNAH,1999-09-15,999.0,X,32.1,-81.1,3.0,1.0,1880-01-01,2387.46,GA,31401
1721816,MANDEVILLE,2002-09-26,999.0,AE,30.4,-90.1,2.0,1.0,1888-12-31,2803.92,LA,70448
196601,WAHPETON,1997-04-10,999.0,X,46.3,-96.6,1.0,1.0,1891-07-01,1642.58,ND,58075
...,...,...,...,...,...,...,...,...,...,...,...,...
518034,MONT BELVIEU,2019-05-09,999.0,X,29.8,-94.8,2.0,1.0,2019-01-01,50839.66,TX,77523
334993,SAINT CHARLES,2019-06-07,999.0,X,38.9,-90.5,2.0,1.0,2019-02-01,944.65,MO,63301
1064006,ORANGE,2019-05-09,999.0,C,30.2,-93.9,1.0,4.0,2019-02-01,49295.16,TX,77632
1129528,MINNEAPOLIS,2019-04-26,999.0,X,45.1,-93.4,2.0,1.0,2019-03-13,0.00,MN,55445


In [10]:
FEMA_Clean14_df.dtypes

reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate              datetime64[ns]
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
dtype: object

### D. Due to processing time, limit the Data to only 3 years (2013-2015) 

In [11]:
FEMA_Clean15_df=FEMA_Clean14_df[FEMA_Clean14_df["dateofloss"] > "2012-12-31"]
FEMA_Clean15_df

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
6,HOUSTON,2017-08-27,999.0,X,29.7,-95.5,1.0,1.0,1974-01-01,195857.43,TX,77096
48,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,MA,2650
84,SCITUATE,2018-03-02,999.0,X,42.2,-70.7,3.0,1.0,1981-07-01,3201.31,MA,2066
85,SCITUATE,2018-06-09,999.0,X,42.2,-70.7,3.0,1.0,1981-07-01,0.00,MA,2066
111,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,TX,78130
...,...,...,...,...,...,...,...,...,...,...,...,...
2432874,KINGWOOD,2017-08-27,3.0,AE,30.1,-95.1,1.0,1.0,2002-01-01,250000.00,TX,77345
2432875,HOUSTON,2017-08-29,999.0,X,29.9,-95.3,1.0,1.0,1971-01-01,101508.49,TX,77039
2432876,SAN ANTONIO,2013-05-26,999.0,X,29.5,-98.5,1.0,1.0,1960-06-01,75882.72,TX,78216
2432877,HOUSTON,2017-08-26,999.0,X,29.8,-95.2,1.0,1.0,1952-06-01,76068.15,TX,77015


In [12]:
FEMA_Clean15_df=FEMA_Clean15_df[FEMA_Clean15_df["dateofloss"] < "2016-01-01"]
FEMA_Clean15_df

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
48,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,MA,2650
111,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,TX,78130
153,PENSACOLA,2014-04-30,2.0,AE,30.3,-87.5,2.0,1.0,1982-07-01,0.00,FL,32507
160,FALLBROOK,2013-08-16,999.0,X,33.4,-117.2,2.0,1.0,1993-01-01,0.00,CA,92028
300,EDISTO BEACH,2015-10-03,5.0,AE,32.5,-80.3,1.0,1.0,1980-01-01,0.00,SC,29438
...,...,...,...,...,...,...,...,...,...,...,...,...
2432870,HOUSTON,2014-05-28,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,2259.89,TX,77080
2432871,HOUSTON,2015-05-25,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,1638.83,TX,77080
2432873,NEW BRAUNFELS,2015-10-30,999.0,X,29.7,-98.2,1.0,1.0,1974-01-01,13929.98,TX,78130
2432876,SAN ANTONIO,2013-05-26,999.0,X,29.5,-98.5,1.0,1.0,1960-06-01,75882.72,TX,78216


In [13]:
FEMA_Clean15_df.sort_values('dateofloss')

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode
2420701,LAKE CHARLES,2013-01-01,0.0,AE,30.2,-93.2,1.0,1.0,1977-01-01,7833.91,LA,70605
306300,PEABODY,2013-01-01,999.0,C,42.5,-71.0,2.0,1.0,2003-01-01,10074.85,MA,1960
262619,TEXAS CITY,2013-01-01,999.0,B,29.4,-94.9,1.0,1.0,1978-07-15,0.00,TX,77590
1729992,CORAL SPRINGS,2013-01-01,2.0,AHB,26.3,-80.2,1.0,1.0,1987-01-01,0.00,FL,33067
742997,ROCHESTER,2013-01-01,999.0,C,46.8,-123.1,5.0,1.0,1984-01-01,0.00,WA,98579
...,...,...,...,...,...,...,...,...,...,...,...,...
599158,DUMAS,2015-12-31,999.0,A,34.0,-91.4,5.0,1.0,1981-04-06,0.00,AR,71639
599366,MAYFLOWER,2015-12-31,0.0,AE,35.0,-92.5,2.0,1.0,1984-09-01,87149.44,AR,72106
599461,LITTLE ROCK,2015-12-31,3.0,A12,34.8,-92.4,2.0,1.0,2011-02-20,2467.51,AR,72223
319577,FENTON,2015-12-31,999.0,AE,38.5,-90.4,2.0,2.0,1960-02-01,88000.00,MO,63026


### D.1. Remove the row in originalconstructiondate that causes error, this does not happen for years 2014 to 2015


In [14]:
#Remove the row in originalconstructiondate that causes error, this does not happen for years 2014 to 2015
# FEMA_Clean15_df=FEMA_Clean14_df.drop(549916)

In [15]:
#convert originalconstructiondate type to datetime
FEMA_Clean15_df['originalconstructiondate'] =  pd.to_datetime(FEMA_Clean15_df['originalconstructiondate'], format='%Y-%m-%d', errors='ignore')
FEMA_Clean15_df.dtypes

C:\Users\hcivi_000.000\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


reportedcity                                  object
dateofloss                            datetime64[ns]
elevationdifference                          float64
floodzone                                     object
latitude                                     float64
longitude                                    float64
numberoffloorsintheinsuredbuilding           float64
occupancytype                                float64
originalconstructiondate              datetime64[ns]
amountpaidonbuildingclaim                    float64
state                                         object
reportedzipcode                               object
dtype: object

## E. Convert FEMA DataFram to GeoDataFrame
This is done to create a "point" for each house that was damaged. Later we can use this point to find the closest weather station to damaged house.

In [16]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

In [17]:
gdf_FEMA = geopandas.GeoDataFrame(
    FEMA_Clean15_df, geometry=geopandas.points_from_xy(FEMA_Clean15_df.longitude, FEMA_Clean15_df.latitude))

In [18]:
gdf_FEMA

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode,geometry
48,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,MA,2650,POINT (-70.00000 41.70000)
111,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,TX,78130,POINT (-98.10000 29.70000)
153,PENSACOLA,2014-04-30,2.0,AE,30.3,-87.5,2.0,1.0,1982-07-01,0.00,FL,32507,POINT (-87.50000 30.30000)
160,FALLBROOK,2013-08-16,999.0,X,33.4,-117.2,2.0,1.0,1993-01-01,0.00,CA,92028,POINT (-117.20000 33.40000)
300,EDISTO BEACH,2015-10-03,5.0,AE,32.5,-80.3,1.0,1.0,1980-01-01,0.00,SC,29438,POINT (-80.30000 32.50000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432870,HOUSTON,2014-05-28,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,2259.89,TX,77080,POINT (-95.50000 29.80000)
2432871,HOUSTON,2015-05-25,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,1638.83,TX,77080,POINT (-95.50000 29.80000)
2432873,NEW BRAUNFELS,2015-10-30,999.0,X,29.7,-98.2,1.0,1.0,1974-01-01,13929.98,TX,78130,POINT (-98.20000 29.70000)
2432876,SAN ANTONIO,2013-05-26,999.0,X,29.5,-98.5,1.0,1.0,1960-06-01,75882.72,TX,78216,POINT (-98.50000 29.50000)


# NASA Data

In [2]:
from pylab import *
import netCDF4
import xarray as xr
import numpy as np

## A. Read Data and Create a 1 combined DataFrame (Limit the data to only 3 years due to long processing time)

The NASA 3 years will match the FEMA 3 years (2013-2015)

### A.1. Read NASA datat in .nc4 format

In [1]:
# ds_2010 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2010.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
# ds_2011 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2011.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
# ds_2012 = xr.open_dataset('NASA/data/daymet_v3_stnsxval_prcp_2012.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
ds_2013 = xr.open_dataset('../NASA/data/daymet_v3_stnsxval_prcp_2013.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
ds_2014 = xr.open_dataset('../NASA/data/daymet_v3_stnsxval_prcp_2014.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
ds_2015 = xr.open_dataset('../NASA/data/daymet_v3_stnsxval_prcp_2015.nc4',drop_variables=["stns","station_id","station_name","stnz","time_bnds", "stnx","stny"])
ds_2015

NameError: name 'xr' is not defined

### A.2. Save data in a DataFrame

In [22]:
# ds_2010 = ds_2010.to_dataframe()
# ds_2011 = ds_2011.to_dataframe()
# ds_2012 = ds_2012.to_dataframe()
ds_2013 = ds_2013.to_dataframe()
ds_2014 = ds_2014.to_dataframe()
ds_2015 = ds_2015.to_dataframe()
ds_2015

obs      pred  stn_lat  stn_lon
stns  time                                                 
0     2015-01-01 12:00:00  0.00  0.000000   22.208 -159.406
      2015-01-02 12:00:00  0.15  0.774545   22.208 -159.406
      2015-01-03 12:00:00  6.02  5.175762   22.208 -159.406
      2015-01-04 12:00:00   NaN       NaN   22.208 -159.406
      2015-01-05 12:00:00  0.00  0.000000   22.208 -159.406
...                         ...       ...      ...      ...
20929 2015-12-27 12:00:00  0.00  0.000000   20.683  -88.200
      2015-12-28 12:00:00  0.00  0.000000   20.683  -88.200
      2015-12-29 12:00:00  0.53  0.000000   20.683  -88.200
      2015-12-30 12:00:00  0.00  0.691709   20.683  -88.200
      2015-12-31 12:00:00   NaN       NaN   20.683  -88.200

[7639450 rows x 4 columns]

### A.3. Reset index

In [23]:
# ds_2010=ds_2010.reset_index()
# ds_2011=ds_2011.reset_index()
# ds_2012=ds_2012.reset_index()
ds_2013=ds_2013.reset_index()
ds_2014=ds_2014.reset_index()
ds_2015=ds_2015.reset_index()
ds_2015

,stns,time,obs,pred,stn_lat,stn_lon
0,0,2015-01-01 12:00:00,0.00,0.000000,22.208,-159.406
1,0,2015-01-02 12:00:00,0.15,0.774545,22.208,-159.406
2,0,2015-01-03 12:00:00,6.02,5.175762,22.208,-159.406
3,0,2015-01-04 12:00:00,NaN,NaN,22.208,-159.406
4,0,2015-01-05 12:00:00,0.00,0.000000,22.208,-159.406
...,...,...,...,...,...,...
7639445,20929,2015-12-27 12:00:00,0.00,0.000000,20.683,-88.200
7639446,20929,2015-12-28 12:00:00,0.00,0.000000,20.683,-88.200
7639447,20929,2015-12-29 12:00:00,0.53,0.000000,20.683,-88.200
7639448,20929,2015-12-30 12:00:00,0.00,0.691709,20.683,-88.200


### A.4. Combine the NASA DataFrames

In [24]:
NASA_Comb = pd.concat([
ds_2013,
ds_2014,
ds_2015], ignore_index=True, sort=False)

## B. Convert timestep column to Date Column

This is necessary, because the FEMA dataset has only the "Data" column, which will be used to merge the data.

In [25]:
NASA_Comb['time'] = pd.to_datetime(NASA_Comb['time'])
NASA_Comb['new_date_column'] = NASA_Comb['time'].dt.date
NASA_Comb['new_date_column'] =  pd.to_datetime(NASA_Comb['new_date_column'], format='%Y-%m-%d')


In [26]:
NASA_Comb

,stns,time,obs,pred,stn_lat,stn_lon,new_date_column
0,0,2013-01-01 12:00:00,0.33,0.450303,22.208,-159.406,2013-01-01
1,0,2013-01-02 12:00:00,0.84,0.660080,22.208,-159.406,2013-01-02
2,0,2013-01-03 12:00:00,0.53,1.542686,22.208,-159.406,2013-01-03
3,0,2013-01-04 12:00:00,NaN,NaN,22.208,-159.406,2013-01-04
4,0,2013-01-05 12:00:00,0.23,0.038655,22.208,-159.406,2013-01-05
...,...,...,...,...,...,...,...
22788770,20929,2015-12-27 12:00:00,0.00,0.000000,20.683,-88.200,2015-12-27
22788771,20929,2015-12-28 12:00:00,0.00,0.000000,20.683,-88.200,2015-12-28
22788772,20929,2015-12-29 12:00:00,0.53,0.000000,20.683,-88.200,2015-12-29
22788773,20929,2015-12-30 12:00:00,0.00,0.691709,20.683,-88.200,2015-12-30


In [27]:
NASA_Comb.dtypes

stns                        int64
time               datetime64[ns]
obs                       float64
pred                      float64
stn_lat                   float64
stn_lon                   float64
new_date_column    datetime64[ns]
dtype: object

## C. Convert DataFrame to GeoDataFrame

In [28]:
gdf_NASA = geopandas.GeoDataFrame(
    NASA_Comb, geometry=geopandas.points_from_xy(NASA_Comb.stn_lon, NASA_Comb.stn_lat))

In [29]:
# gdf_NASA.to_csv('gdf_NASA.csv')

In [30]:
# gdf_NASA=pd.read_csv('gdf_NASA.csv')
# gdf_NASA.head(15)

## D. Convert all nan Data on "obs" Column to "0"

obs represents the amout the rainfall that the weather station has recorded on a specific date
Many obs data was not recorded and the cell was empty. Therefore it was assumed that there was no rain on that specific date. We do underestand that this may cause some inaccuracy. 

In [31]:
gdf_NASA['obs'] = gdf_NASA['obs'].fillna(0.0)
gdf_NASA

,stns,time,obs,pred,stn_lat,stn_lon,new_date_column,geometry
0,0,2013-01-01 12:00:00,0.33,0.450303,22.208,-159.406,2013-01-01,POINT (-159.40600 22.20800)
1,0,2013-01-02 12:00:00,0.84,0.660080,22.208,-159.406,2013-01-02,POINT (-159.40600 22.20800)
2,0,2013-01-03 12:00:00,0.53,1.542686,22.208,-159.406,2013-01-03,POINT (-159.40600 22.20800)
3,0,2013-01-04 12:00:00,0.00,NaN,22.208,-159.406,2013-01-04,POINT (-159.40600 22.20800)
4,0,2013-01-05 12:00:00,0.23,0.038655,22.208,-159.406,2013-01-05,POINT (-159.40600 22.20800)
...,...,...,...,...,...,...,...,...
22788770,20929,2015-12-27 12:00:00,0.00,0.000000,20.683,-88.200,2015-12-27,POINT (-88.20000 20.68300)
22788771,20929,2015-12-28 12:00:00,0.00,0.000000,20.683,-88.200,2015-12-28,POINT (-88.20000 20.68300)
22788772,20929,2015-12-29 12:00:00,0.53,0.000000,20.683,-88.200,2015-12-29,POINT (-88.20000 20.68300)
22788773,20929,2015-12-30 12:00:00,0.00,0.691709,20.683,-88.200,2015-12-30,POINT (-88.20000 20.68300)


# Combine FEMA and NASA Datasets

## A. Create a Function to find the closest distance between the points between FEMA and NASA Data, and Merge the 2 datasets

The function creates a new column called "dist", which represents the disctance between the 2 "Points" from each dataset.

In [32]:
import geopandas as gpd
import numpy as np
import pandas as pd

from scipy.spatial import cKDTree
from shapely.geometry import Point

gpd1 = gpd.GeoDataFrame([['John', 1, Point(41.7, -70.0)]],
                        
                        columns=['Name', 'ID', 'geometry'])
gpd2 = gpd.GeoDataFrame([['Work', Point(41.6, -69.99)]],
                        
                        columns=['Place', 'geometry'])

def ckdnearest(gdA, gdB):
    nA = np.array(list(zip(gdA.geometry.x, gdA.geometry.y)) )
    nB = np.array(list(zip(gdB.geometry.x, gdB.geometry.y)) )
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdf = pd.concat(
        [gdA.reset_index(drop=True), gdB.loc[idx, gdB.columns != 'geometry'].reset_index(drop=True),
         pd.Series(dist, name='dist')], axis=1)
    return gdf

ckdnearest(gpd1, gpd2)

,Name,ID,geometry,Place,dist
0,John,1,POINT (41.70000 -70.00000),Work,0.100499


In [33]:
import math
math.sqrt((0.01)**2 + 0.1 **2)

0.1004987562112089

In [34]:
#Run the function
gdf_FEMA_plus_station_id=ckdnearest(gdf_FEMA, gdf_NASA) #gdf_NASA[:10000])

## B. Merge FEMA and NASA datasets

The fist merged dataset is created based on the closest distance between points. 
From this merge we can find the closest wether station to each damaged hourse. See column "stns"
But the function doesn not look at matching the date from FEMA "dateofloss" and the date from NASA "new_date_column"

Considering we were able to match the weather station "stns" from NASA to FEMA database, we need to do the following:

### B.1. Remove Columns from "first merged" dataset that does not have correct value "time", "obs", "pred", "stn_lat", "stn_lon", "new_date_column", "dist", "geometry". 

### B.2. Keep Station column "stns" in the "first merged" dataset 

### B.3 Calculate the comulative amount of rain in the last 2 days and save it in a new column

### B.4. Merge NASA dataset again with "first merged" dataset based on the Station and Date

In [35]:
gdf_FEMA_plus_station_id.head()

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,...,reportedzipcode,geometry,stns,time,obs,pred,stn_lat,stn_lon,new_date_column,dist
0,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,...,2650,POINT (-70.00000 41.70000),14281,2015-01-31 12:00:00,0.13,0.233623,41.6875,-69.9933,2015-01-31,0.014182
1,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,...,78130,POINT (-98.10000 29.70000),526,2013-10-21 12:00:00,0.00,0.000000,29.7044,-98.0941,2013-10-21,0.007360
2,PENSACOLA,2014-04-30,2.0,AE,30.3,-87.5,2.0,1.0,1982-07-01,0.00,...,32507,POINT (-87.50000 30.30000),2048,2015-12-30 12:00:00,0.28,0.464730,30.3130,-87.4672,2015-12-30,0.035282
3,FALLBROOK,2013-08-16,999.0,X,33.4,-117.2,2.0,1.0,1993-01-01,0.00,...,92028,POINT (-117.20000 33.40000),2462,2013-12-30 12:00:00,0.00,0.000000,33.4392,-117.1900,2013-12-30,0.040455
4,EDISTO BEACH,2015-10-03,5.0,AE,32.5,-80.3,1.0,1.0,1980-01-01,0.00,...,29438,POINT (-80.30000 32.50000),4024,2013-10-16 12:00:00,0.00,0.000000,32.5050,-80.2947,2013-10-16,0.007286


### B.1. Remove Columns from "first merged" dataset that does not have correct value.
### B.2. Keep Station column "stns" in the "first merged" dataset 

In [36]:
gdf_FEMA_plus_station_id=gdf_FEMA_plus_station_id.drop(columns=["time","obs","pred","stn_lat","stn_lon","new_date_column","dist","geometry"])

In [37]:
gdf_FEMA_plus_station_id

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,state,reportedzipcode,stns
0,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,MA,2650,14281
1,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,TX,78130,526
2,PENSACOLA,2014-04-30,2.0,AE,30.3,-87.5,2.0,1.0,1982-07-01,0.00,FL,32507,2048
3,FALLBROOK,2013-08-16,999.0,X,33.4,-117.2,2.0,1.0,1993-01-01,0.00,CA,92028,2462
4,EDISTO BEACH,2015-10-03,5.0,AE,32.5,-80.3,1.0,1.0,1980-01-01,0.00,SC,29438,4024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75732,HOUSTON,2014-05-28,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,2259.89,TX,77080,832
75733,HOUSTON,2015-05-25,999.0,X,29.8,-95.5,1.0,1.0,1972-06-01,1638.83,TX,77080,832
75734,NEW BRAUNFELS,2015-10-30,999.0,X,29.7,-98.2,1.0,1.0,1974-01-01,13929.98,TX,78130,515
75735,SAN ANTONIO,2013-05-26,999.0,X,29.5,-98.5,1.0,1.0,1960-06-01,75882.72,TX,78216,510


### B.3 Calculate the comulative amount of rain in the last 2 days and save it in a new column

In [38]:
gdf_NASA['rolling_2days_obs'] = gdf_NASA['obs'].rolling(2).sum()
gdf_NASA['rolling_7days_obs'] = gdf_NASA['obs'].rolling(7).sum()

### B.4. Merge NASA dataset again with "first merged" dataset based on the Station and Date

In [39]:
merged_df = pd.merge(gdf_FEMA_plus_station_id, gdf_NASA, left_on=['stns', 'dateofloss'], right_on=['stns', 'new_date_column'])

In [40]:
merged_df.keys()

Index(['reportedcity', 'dateofloss', 'elevationdifference', 'floodzone',
       'latitude', 'longitude', 'numberoffloorsintheinsuredbuilding',
       'occupancytype', 'originalconstructiondate',
       'amountpaidonbuildingclaim', 'state', 'reportedzipcode', 'stns', 'time',
       'obs', 'pred', 'stn_lat', 'stn_lon', 'new_date_column', 'geometry',
       'rolling_2days_obs', 'rolling_7days_obs'],
      dtype='object')

## C. Combined NASA and FEMA Dataset

In [41]:
merged_df

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,...,stns,time,obs,pred,stn_lat,stn_lon,new_date_column,geometry,rolling_2days_obs,rolling_7days_obs
0,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,...,14281,2015-02-21 12:00:00,0.38,0.000000,41.6875,-69.9933,2015-02-21,POINT (-69.99330 41.68750),3.800000e-01,1.680000e+00
1,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,...,526,2015-10-30 12:00:00,10.62,9.319764,29.7221,-98.1311,2015-10-30,POINT (-98.13110 29.72210),1.062000e+01,2.208000e+01
2,NEW BRAUNFELS,2015-10-30,7.0,AE,29.7,-98.1,2.0,1.0,2003-07-01,0.00,...,526,2015-10-30 12:00:00,10.62,9.319764,29.7221,-98.1311,2015-10-30,POINT (-98.13110 29.72210),1.062000e+01,2.208000e+01
3,NEW BRAUNFELS,2015-10-30,0.0,C,29.7,-98.1,3.0,1.0,1998-10-17,67301.33,...,526,2015-10-30 12:00:00,10.62,9.319764,29.7221,-98.1311,2015-10-30,POINT (-98.13110 29.72210),1.062000e+01,2.208000e+01
4,NEW BRAUNFELS,2015-10-30,999.0,C,29.7,-98.1,3.0,3.0,1995-07-01,0.00,...,526,2015-10-30 12:00:00,10.62,9.319764,29.7221,-98.1311,2015-10-30,POINT (-98.13110 29.72210),1.062000e+01,2.208000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75656,HAMMOND,2014-04-11,999.0,A,44.5,-75.6,2.0,1.0,1976-01-01,0.00,...,17787,2014-04-11 12:00:00,0.00,0.000000,44.3768,-69.2616,2014-04-11,POINT (-69.26160 44.37680),8.216761e-13,1.120000e+00
75657,ARVERNE,2014-01-08,999.0,A05,40.6,-73.8,3.0,1.0,1920-01-01,0.00,...,14209,2014-01-08 12:00:00,0.10,0.343227,42.5420,-123.4260,2014-01-08,POINT (-123.42600 42.54200),4.600000e-01,4.600000e-01
75658,YORKTOWN HTS,2013-06-08,999.0,X,41.3,-73.8,4.0,1.0,1968-04-01,3198.97,...,14262,2013-06-08 12:00:00,8.64,6.556328,41.2664,-73.7975,2013-06-08,POINT (-73.79750 41.26640),1.042000e+01,1.205000e+01
75659,MC GREGOR,2013-10-26,999.0,X,31.5,-97.3,1.0,1.0,2000-06-28,12654.47,...,1561,2013-10-26 12:00:00,0.00,0.000000,30.6081,-97.0317,2013-10-26,POINT (-97.03170 30.60810),-2.409184e-14,1.109113e-13


In [42]:
merged_clean_df=merged_df.drop(columns=["time"])
merged_clean_df

,reportedcity,dateofloss,elevationdifference,floodzone,latitude,longitude,numberoffloorsintheinsuredbuilding,occupancytype,originalconstructiondate,amountpaidonbuildingclaim,...,reportedzipcode,stns,obs,pred,stn_lat,stn_lon,new_date_column,geometry,rolling_2days_obs,rolling_7days_obs
0,NO CHATHAM,2015-02-21,999.0,AE,41.7,-70.0,3.0,1.0,1955-01-01,2618.18,...,2650,14281,0.38,0.000000,41.6875,-69.9933,2015-02-21,POINT (-69.99330 41.68750),3.800000e-01,1.680000e+00
1,NEW BRAUNFELS,2015-10-30,2.0,AE,29.7,-98.1,3.0,1.0,2003-12-02,3309.59,...,78130,526,10.62,9.319764,29.7221,-98.1311,2015-10-30,POINT (-98.13110 29.72210),1.062000e+01,2.208000e+01
2,NEW BRAUNFELS,2015-10-30,7.0,AE,29.7,-98.1,2.0,1.0,2003-07-01,0.00,...,78130,526,10.62,9.319764,29.7221,-98.1311,2015-10-30,POINT (-98.13110 29.72210),1.062000e+01,2.208000e+01
3,NEW BRAUNFELS,2015-10-30,0.0,C,29.7,-98.1,3.0,1.0,1998-10-17,67301.33,...,78132,526,10.62,9.319764,29.7221,-98.1311,2015-10-30,POINT (-98.13110 29.72210),1.062000e+01,2.208000e+01
4,NEW BRAUNFELS,2015-10-30,999.0,C,29.7,-98.1,3.0,3.0,1995-07-01,0.00,...,78130,526,10.62,9.319764,29.7221,-98.1311,2015-10-30,POINT (-98.13110 29.72210),1.062000e+01,2.208000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75656,HAMMOND,2014-04-11,999.0,A,44.5,-75.6,2.0,1.0,1976-01-01,0.00,...,13646,17787,0.00,0.000000,44.3768,-69.2616,2014-04-11,POINT (-69.26160 44.37680),8.216761e-13,1.120000e+00
75657,ARVERNE,2014-01-08,999.0,A05,40.6,-73.8,3.0,1.0,1920-01-01,0.00,...,11692,14209,0.10,0.343227,42.5420,-123.4260,2014-01-08,POINT (-123.42600 42.54200),4.600000e-01,4.600000e-01
75658,YORKTOWN HTS,2013-06-08,999.0,X,41.3,-73.8,4.0,1.0,1968-04-01,3198.97,...,10598,14262,8.64,6.556328,41.2664,-73.7975,2013-06-08,POINT (-73.79750 41.26640),1.042000e+01,1.205000e+01
75659,MC GREGOR,2013-10-26,999.0,X,31.5,-97.3,1.0,1.0,2000-06-28,12654.47,...,76657,1561,0.00,0.000000,30.6081,-97.0317,2013-10-26,POINT (-97.03170 30.60810),-2.409184e-14,1.109113e-13


In [43]:
merged_clean_df.to_csv('merged_clean_3Years_df.csv')